### Graham Kerr
#### graham.s.kerr@NASA.gov; kerrg@cua.edu

<H1 font size="+3" style="color:red">
OZ project Project<br>
-- Scratch Notebook 
</H1>

<b>This notebook is just used for testing random bits of code and experimenting</b>





---
### <b style="color:blue"> Some set up </b>

***Import Modules***

In [2]:
##
## Import some modules
##

from scipy import io
from scipy import constants as constants
import sys
sys.path.insert(0,'/Users/gskerr1/Documents/Research/Python_Programs/radynpy/')
sys.path.insert(0,'/Users/gskerr1/Documents/Research/OrrallZirkerEffect/')


import radynpy
from radynpy.utils import RadynMovie as RM
import OrrallZirkerPy as OZpy
import OrrallZirkerPy.Atmos as Atmos
import OrrallZirkerPy.EnergyToVel as e2v
from OrrallZirkerPy.CrossSecActive import CSec_H3lev as H3lev
from OrrallZirkerPy.CrossSecActive import CSec_H2lev as H2lev
from OrrallZirkerPy.CrossSections import chebyshev_fn as Cheb
from numpy.polynomial.polynomial import Polynomial as Poly


import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
from matplotlib.ticker import LogLocator
from matplotlib import ticker
import matplotlib.colorbar as cb
import pandas as pd

import cmocean
import colorcet as ccet
import palettable as pal 
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [33]:
import importlib
importlib.reload(OZpy.CrossSections)
importlib.reload(OZpy.CrossSecActive)
importlib.reload(OZpy.Atmos)
importlib.reload(OZpy.EnergyToVel)


<module 'OrrallZirkerPy.EnergyToVel' from '/Users/gskerr1/Documents/Research/OrrallZirkerEffect/OrrallZirkerPy/EnergyToVel.py'>

In [34]:
##
## Plot properties (these are just personal preference)
##

#Avenir LT Std
font = {'family': 'Avenir LT Std',
        'color':  'black',
        'weight': 'medium',
        'size': 22,
        }
plot_params = {'ytick.direction': 'in', 
               'xtick.direction': 'in', 
               'xtick.minor.visible': True,
               'ytick.minor.visible': True,
               'xtick.major.size': 10, 'xtick.minor.size': 5,
               'ytick.major.size': 10, 'ytick.minor.size': 5,
               'ytick.right': True,
               'xtick.top': True,
               'ytick.major.width': 1.5,
               'xtick.major.width': 1.5,
               'ytick.minor.width': 1.5,
               'xtick.minor.width': 1.5,
               'axes.linewidth': 1.5,
               'axes.spines.top': True,
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.titlepad' : 18 }

plot_lg_params = {'legend.frameon': False}
#plt.rcParams.update({'font.size': font['size'], 'font.family':font['family'], 'font.weight':font['weight'], 'font.color':font['color']})

plt.rcParams.update({'font.size':font['size'], 'font.family':font['family'], 'font.weight':font['weight']})
plt.rcParams.update({'ytick.direction': plot_params['ytick.direction'],
                     'xtick.direction': plot_params['xtick.direction'],
                     'xtick.minor.visible': plot_params['xtick.minor.visible'],
                     'ytick.minor.visible': plot_params['ytick.minor.visible'],
                     'ytick.major.size':  plot_params['ytick.major.size'], 
                     'ytick.minor.size':  plot_params['ytick.minor.size'],
                     'xtick.major.size':  plot_params['xtick.major.size'],                                
                     'xtick.minor.size':  plot_params['xtick.minor.size'],
                     'ytick.right': plot_params['ytick.right'],
                     'xtick.top': plot_params['xtick.top'],
                     'ytick.major.width': plot_params['ytick.major.width'],
                     'xtick.major.width': plot_params['xtick.major.width'],
                     'ytick.minor.width': plot_params['ytick.minor.width'],
                     'xtick.minor.width': plot_params['xtick.minor.width'],                    
                     'axes.linewidth': plot_params['axes.linewidth'],
                     'axes.spines.top' : plot_params['axes.spines.top'],
                     'axes.spines.bottom' : plot_params['axes.spines.bottom'],
                     'axes.spines.left' : plot_params['axes.spines.left'],
                     'axes.spines.right' : plot_params['axes.spines.right'],
                     'axes.titlepad' : plot_params['axes.titlepad'],
                    })

plt.rcParams.update({'legend.frameon': plot_lg_params['legend.frameon']})

mpl.mathtext.SHRINK_FACTOR = 0.6
mpl.mathtext.GROW_FACTOR = 1 / 0.6





template = dict(
        layout = go.Layout(font = dict(family = "Rockwell", size = 18),
                           title_font = dict(family = "Rockwell", size = 22), 
                           plot_bgcolor = 'white',
                           paper_bgcolor = 'white',
                           xaxis = dict(
                                showexponent = 'all',
                                exponentformat = 'e',
                                tickangle = 0,
                                linewidth = 4,
                                showgrid = True,
                            ),
                            yaxis = dict(
                          showexponent = 'all',
                          exponentformat = 'e',
                                linewidth = 4,
                                showgrid = True,
                                anchor = 'free',
                                position = 0,
                                domain = [0.0,1]
                            ),
                            coloraxis_colorbar = dict(
                                thickness = 15,
                                tickformat = '0.2f',
                                ticks = 'outside',
                                titleside = 'right'
                            )
                            ))

---
---
### <b style="color:blue"> Setting the 'active' cross sections </b>

<b>CrossSecActive.py should be modified if you want to select different cross sections. It extracts them from CrossSections.py, so they have to be located there first.</b>
> - Set up an energy range. 
> - Create the csec object to extract *all* cross sections. From this, you can create objects holding cross sections from the various sources.<br>
> - Create the active set object from the set list (either H3lev, or H2lev)... this ***does not*** require the OZpy.CrossSections.CrossSec object. <b style="color:red"> Is this confusing, should I rename some things?</b>

In [35]:
energy = np.arange(1,8001,1)
csec = OZpy.CrossSections.CrossSec(energy)
kerr = csec.cs_kerr_poly()
csactive_3lev = H3lev(energy)
csactive_2lev = H2lev(energy)



In [36]:
csactive_3lev.Q_p1


array([1.68633033e+02, 1.44360965e+02, 1.23088578e+02, ...,
       1.17513140e-10, 1.17433655e-10, 1.17354237e-10])

In [37]:
csecA = OZpy.CrossSecActive.CSecActive(energy, nLev = 1)


>>> You have not entered a valid value of nLev (nLev = 2 or 3)
     Defaulting to nLev = 3!


In [39]:
csecA.nLev_plus_cont

4

---
---
### <b style="color:blue"> Read in a RADYN atmosphere and create the atmos object </b>
> - Does not need to be a RADYN atmosphere, can be anything that gives you hydrogen, electron, and proton number densities.<br>


In [ ]:
###
### Read the RADYN atmosphere
###
dir1 = '/Users/gskerr1/Documents/Research/OrrallZirkerEffect/protonbeam_simulations/F11/'
# cdf1 = radynpy.cdf.LazyRadynData(dir1+'QSHTSL_0.75F10_d7_ec20_t30s_oneD_ne200_ext.cdf')
file1 = 'radyn_out.1000keV.5.F11.sav'

RADYN = io.readsav(dir1+file1, python_dict=True)
# RADYN = RADYN['radyn_struct']


In [ ]:
nElec = RADYN['ne1t']
nHyd = np.sum(RADYN['n1t'][:,0,0:4,:],axis=1)
nProt = RADYN['n1t'][:,0,5,:]
height = RADYN['z1t']
time = RADYN['timet']


In [ ]:
plt.plot(height[0,:]/1e8, nElec[0,:])
plt.plot(height[0,:]/1e8, nProt[0,:])
plt.plot(height[0,:]/1e8, nHyd[0,:])
plt.yscale('log')


In [ ]:
atmo = Atmos.ambientpops(nHyd = nHyd, nElec=nElec, nProt=nProt, height=height, times=time)


---
---
### <b style="color:blue"> Convert energy to velocity </b>
> - Converts energy in keV to particle velocity (assumes proton).<br>

In [ ]:
vels = e2v.energy2vel(energy)


In [ ]:
plt.plot(energy, vels/1e5)
# plt.axhline(y=3e5)
plt.xscale('log')
